In [68]:
# -*- coding: utf-8 -*-
"""
Predicitve_Analytics.py
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats
import time

In [69]:
df = import_data('data.csv')

input_data = df.values

np.random.shuffle(input_data)

X = input_data[:, :input_data.shape[1]-1]
y = input_data[:, input_data.shape[1]-1]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape[1]

48

In [23]:
k = 10

In [ ]:
def find_center(cluster_centers, row, k):
    
    rows = np.tile(row, (k,1))
    
    diff_vector = np.sum(np.absolute(cluster_centers - rows), axis = 1)
    
    return np.argmin(diff_vector)

In [ ]:
def Accuracy(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    
    """

In [ ]:
def Recall(y_true,y_pred):
     """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """

In [ ]:
def Precision(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """
    

In [13]:
def WCSS(Clusters):
    """
    :Clusters List[numpy.ndarray]
    """
    

    

In [ ]:
def ConfusionMatrix(y_true,y_pred):
    
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """  

In [93]:
def KNN(X_train,X_test,Y_train):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """
    k=5
    labels = np.zeros(X_test.shape[0])
    for x in range(X_test.shape[0]):
        row = np.tile(X_test[x], (X_train.shape[0],1))
        diff_matrix = np.mean(np.absolute(X_train - row), axis=1)
        labelled_matrix = np.vstack((diff_matrix, Y_train))
        labelled_matrix = labelled_matrix.T
        sorted_matrix = labelled_matrix[labelled_matrix[:,0].argsort()]
        top_k = sorted_matrix[:k]
        labels[x] = stats.mode(top_k[:,1])[0]
    return labels

In [ ]:
def RandomForest(X_train,Y_train,X_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """

In [ ]:
def PCA(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: numpy.ndarray
    """
 

In [7]:
   
def Kmeans(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: List[numpy.ndarray]
    """
    
    cluster_centers = X_train[np.random.choice(X_train.shape[0], size = N), ]
    clusters = [np.zeros((1,X_train.shape[1])) for x in range(N)]
    
    
    for x in range(X_train.shape[0]):
        center_index = find_center(cluster_centers, X_train[x], N)
        
        # Adding the new point to the cluster.        
        clusters[center_index] = np.vstack((clusters[center_index], X_train[x]))
        
        # Recomputing the center.
        cluster_centers[center_index] = np.mean(clusters[center_index], axis = 0)
        
    
    return clusters
        
        
    
    
    
    
    
    

In [ ]:
def SklearnSupervisedLearning(X_train,Y_train,X_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """

In [ ]:
def SklearnVotingClassifier(X_train,Y_train,X_test):
    
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """

In [3]:
"""
Create your own custom functions for Matplotlib visualization of hyperparameter search. 
Make sure that plots are labeled and proper legends are used
"""

def import_data(filename):
    df = pd.read_csv(filename)
    return df

    

In [12]:
start_time = time.time()
z = Kmeans(X_train, k)
WCSSC(z)

print("--- %s seconds ---" % (time.time() - start_time))

--- 29.562199115753174 seconds ---


In [94]:
start_time = time.time()
z= KNN(X_train, X_test, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

[ 8. 10.  7. ...  2.  4.  5.]

--- 128.1010763645172 seconds ---


In [74]:
z.shape


(8192,)

In [75]:
print(X_test.shape)

(8192, 48)


[ 7.2506e-06  2.3415e-05 -4.1742e-04 -2.4040e-06  1.2904e-05 -1.4468e-04
 -3.0434e-02 -3.0458e-02 -3.0040e-02 -3.8697e-02 -3.8710e-02 -3.8565e-02
  2.2615e-03  1.1658e-03  3.8791e-03  2.2424e-03  1.1607e-03  3.6807e-03
  1.3772e+00  1.3772e+00  1.3765e+00  1.3829e+00  1.3829e+00  1.3820e+00
 -1.5663e-02  1.8253e-01 -1.8392e+00 -4.6197e-03  6.9099e-02 -3.4398e-01
  6.7915e-03  6.7470e-03  7.0187e-03 -1.8585e-02 -1.8595e-02 -1.8903e-02
 -7.3678e-01  6.1845e-01  1.7398e+01 -7.4332e-01  6.9332e-01  2.4694e+00
 -1.5027e+00 -1.5027e+00 -1.5025e+00 -1.4896e+00 -1.4896e+00 -1.4897e+00]
